In [376]:
import os
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
import os
import torchvision.models as models
import torch.nn as nn
from deepproblog.dataset import Dataset, DataLoader
from torch.utils.data import Dataset as TorchDataset
from problog.logic import Term, Constant, Var

import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import numpy as np
from mtcnn import MTCNN
import random
from deepproblog.utils.stop_condition import EpochStop
from deepproblog.optimizer import SGD

from deepproblog.model import Model
from deepproblog.network import Network
from deepproblog.engines import ExactEngine
from deepproblog.query import Query
from deepproblog.train import train_model


In [377]:
# GLOBAL VARS
training_size_perc = 1
epochs = 2
lr_scene_reduction = 0.0001
lr_faces = 0.0001
lr_scenes = 0.0001
lr_model = 0.0001
batch_size = 16

In [378]:
# Directories
script_dir = os.getcwd()
print("script_dir:", script_dir)
base_dir = os.path.dirname(os.path.dirname(script_dir))
print("base_dir:", base_dir)
data_dir = os.path.join(base_dir, "data")
print("data_dir:", data_dir)
findingemo_dir = os.path.join(data_dir, "FindingEmo_Images")
print("findingemo_dir:", findingemo_dir)
prolog_dir = os.path.join(base_dir, "code", "prolog")
print("prolog_dir:", prolog_dir)
df = pd.read_pickle(os.path.join(data_dir, "dataframe_cleaned.pkl"))

script_dir: c:\Users\jarne\Documents\Code Masterthesis\nsai_social_cognition\code\notebooks
base_dir: c:\Users\jarne\Documents\Code Masterthesis\nsai_social_cognition
data_dir: c:\Users\jarne\Documents\Code Masterthesis\nsai_social_cognition\data
findingemo_dir: c:\Users\jarne\Documents\Code Masterthesis\nsai_social_cognition\data\FindingEmo_Images
prolog_dir: c:\Users\jarne\Documents\Code Masterthesis\nsai_social_cognition\code\prolog


In [379]:
emotion_categories_hf = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

EMOTION_SETS = [["Serenity", "Joy", "Ecstasy"],
                    ["Acceptance", "Trust", "Admiration"],
                    ["Apprehension", "Fear", "Terror"],
                    ["Distraction", "Surprise", "Amazement"],
                    ["Pensiveness", "Sadness", "Grief"],
                    ["Boredom", "Disgust", "Loathing"],
                    ["Annoyance", "Anger", "Rage"],
                    ["Interest", "Anticipation", "Vigilance"]]

EMO8_LIST = [l[1] for l in EMOTION_SETS]

In [380]:
# Apply the mapping to 8 emotions as defined in the FindingEmo code
EMO8_MAPPING = {}

for i, leaf in enumerate(EMOTION_SETS):
    for emo in leaf:
        EMO8_MAPPING[emo] = EMO8_LIST[i]

In [ ]:
# get the different emo8's  
basic_6_emotions = ["Anger", "Disgust", "Fear", "Joy", "Sadness", "Surprise"]
basic_6_emotions_idx = {emo: i for i, emo in enumerate(basic_6_emotions)}

In [ ]:
basic_6_emotions_idx

{'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Joy': 3, 'Sadness': 4, 'Surprise': 5}

In [ ]:
# Load category names
scene_categories_path = os.path.join(data_dir, "places365/categories_places365.txt")
with open(scene_categories_path) as f:
    scene_categories = [line.strip().split(' ')[0][3:] for line in f]
    # If there is a / in the category name, replace it with _
    scene_categories = [cat.replace('/', '_') for cat in scene_categories]

In [ ]:
places365_to_13categories = {
    # --- Transport infrastructure ---
    "airfield": "transport_infrastructure",
    "airplane_cabin": "transport_infrastructure",
    "airport_terminal": "transport_infrastructure",
    "railroad_track": "transport_infrastructure",
    "train_interior": "transport_infrastructure",
    "train_station_platform": "transport_infrastructure",
    "subway_station_platform": "transport_infrastructure",
    "bridge": "transport_infrastructure",
    "runway": "transport_infrastructure",
    "highway": "transport_infrastructure",
    "garage_indoor": "transport_infrastructure",
    "garage_outdoor": "transport_infrastructure",
    "bus_interior": "transport_infrastructure",
    "bus_station_indoor": "transport_infrastructure",
    "harbor": "transport_infrastructure",
    "viaduct": "transport_infrastructure",
    "heliport": "transport_infrastructure",
    "parking_garage_indoor": "transport_infrastructure",
    "parking_garage_outdoor": "transport_infrastructure",
    "car_interior": "transport_infrastructure",
    "cockpit": "transport_infrastructure",
    "berth": "transport_infrastructure",

    # --- Indoor residential ---
    "bedroom": "indoor_residential",
    "bathroom": "indoor_residential",
    "living_room": "indoor_residential",
    "kitchen": "indoor_residential",
    "attic": "indoor_residential",
    "childs_room": "indoor_residential",
    "closet": "indoor_residential",
    "dining_room": "indoor_residential",
    "dorm_room": "indoor_residential",
    "hotel_room": "indoor_residential",
    "mansion": "indoor_residential",
    "nursery": "indoor_residential",
    "television_room": "indoor_residential",
    "pantry": "indoor_residential",
    "playroom": "indoor_residential",
    "house": "indoor_residential",
    "apartment_building_outdoor": "indoor_residential",
    "balcony_interior": "indoor_residential",
    "banquet_hall": "indoor_residential",
    "porch": "indoor_residential",
    "jacuzzi_indoor": "indoor_residential",
    "shower": "indoor_residential",
    "alcove": "indoor_residential",
    "bedchamber": "indoor_residential",
    "ballroom": "indoor_residential",
    "basement": "indoor_residential",
    "dressing_room": "indoor_residential",
    "home_office": "indoor_residential",
    "home_theater": "indoor_residential",
    "manufactured_home": "indoor_residential",
    "utility_room": "indoor_residential",
    "wet_bar": "indoor_residential",
    "youth_hostel": "indoor_residential",

    # --- Indoor commercial ---
    "bakery_shop": "indoor_commercial",
    "bar": "indoor_commercial",
    "bookstore": "indoor_commercial",
    "butchers_shop": "indoor_commercial",
    "candy_store": "indoor_commercial",
    "clothing_store": "indoor_commercial",
    "department_store": "indoor_commercial",
    "fastfood_restaurant": "indoor_commercial",
    "florist_shop_indoor": "indoor_commercial",
    "gift_shop": "indoor_commercial",
    "jewelry_shop": "indoor_commercial",
    "market_indoor": "indoor_commercial",
    "pet_shop": "indoor_commercial",
    "pharmacy": "indoor_commercial",
    "shoe_shop": "indoor_commercial",
    "shopping_mall_indoor": "indoor_commercial",
    "supermarket": "indoor_commercial",
    "toyshop": "indoor_commercial",
    "restaurant": "indoor_commercial",
    "restaurant_kitchen": "indoor_commercial",
    "restaurant_patio": "indoor_commercial",
    "pizzeria": "indoor_commercial",
    "pub_indoor": "indoor_commercial",
    "sushi_bar": "indoor_commercial",
    "hotel_outdoor": "indoor_commercial",
    "amusement_arcade": "indoor_commercial",
    "auto_showroom": "indoor_commercial",
    "bakery_shop": "indoor_commercial",
    "bank_vault": "indoor_commercial",
    "bazaar_indoor": "indoor_commercial",
    "beauty_salon": "indoor_commercial",
    "cafeteria": "indoor_commercial",
    "coffee_shop": "indoor_commercial",
    "delicatessen": "indoor_commercial",
    "discotheque": "indoor_commercial",
    "drugstore": "indoor_commercial",
    "fabric_store": "indoor_commercial",
    "flea_market_indoor": "indoor_commercial",
    "food_court": "indoor_commercial",
    "hardware_store": "indoor_commercial",
    "laundromat": "indoor_commercial",
    "market_outdoor": "indoor_commercial",
    "motel": "indoor_commercial",
    "pub_indoor": "indoor_commercial",
    "sauna": "indoor_commercial",
    "server_room": "indoor_commercial",
    "shoe_shop": "indoor_commercial",
    "shopping_mall_indoor": "indoor_commercial",
    "supermarket": "indoor_commercial",
    "booth_indoor": "indoor_commercial",
    "ice_cream_parlor": "indoor_commercial",
    "general_store_indoor": "indoor_commercial",
    "general_store_outdoor": "indoor_commercial",
    "pavilion": "indoor_commercial",
    "ticket_booth": "indoor_commercial",

    # --- Indoor institutional ---
    "art_gallery": "indoor_institutional",
    "auditorium": "indoor_institutional",
    "church_indoor": "indoor_institutional",
    "classroom": "indoor_institutional",
    "conference_room": "indoor_institutional",
    "hospital": "indoor_institutional",
    "hospital_room": "indoor_institutional",
    "library_indoor": "indoor_institutional",
    "lecture_room": "indoor_institutional",
    "office": "indoor_institutional",
    "office_cubicles": "indoor_institutional",
    "schoolhouse": "indoor_institutional",
    "science_museum": "indoor_institutional",
    "nursing_home": "indoor_institutional",
    "reception": "indoor_institutional",
    "waiting_room": "indoor_institutional",
    "museum_indoor": "indoor_institutional",
    "biology_laboratory": "indoor_institutional",
    "chemistry_lab": "indoor_institutional",
    "clean_room": "indoor_institutional",
    "conference_center": "indoor_institutional",
    "courthouse": "indoor_institutional",
    "embassy": "indoor_institutional",
    "entrance_hall": "indoor_institutional",
    "elevator_lobby": "indoor_institutional",
    "elevator_shaft": "indoor_institutional",
    "elevator_door": "indoor_institutional",
    "escalator_indoor": "indoor_institutional",
    "hospital": "indoor_institutional",
    "kindergarden_classroom": "indoor_institutional",
    "legislative_chamber": "indoor_institutional",
    "library_outdoor": "indoor_institutional",
    "lobby": "indoor_institutional",
    "locker_room": "indoor_institutional",
    "operating_room": "indoor_institutional",
    "physics_laboratory": "indoor_institutional",
    "recreation_room": "indoor_institutional",
    "veterinarians_office": "indoor_institutional",
    "atrium_public": "indoor_institutional",
    "corridor": "indoor_institutional",
    "computer_room": "indoor_institutional",
    "mezzanine": "indoor_institutional",
    "office_building": "indoor_institutional",
    "jail_cell": "indoor_institutional",
    "storage_room": "indoor_institutional",

    # --- Industrial facilities ---
    "assembly_line": "industrial_facility",
    "auto_factory": "industrial_facility",
    "engine_room": "industrial_facility",
    "industrial_area": "industrial_facility",
    "oilrig": "industrial_facility",
    "construction_site": "industrial_facility",
    "army_base": "industrial_facility",
    "junkyard": "industrial_facility",
    "repair_shop": "industrial_facility",
    "loading_dock": "industrial_facility",
    "fire_station": "industrial_facility",
    "landing_deck": "industrial_facility",
    "construction_site": "industrial_facility",
    "gas_station": "industrial_facility",
    "hangar_indoor": "industrial_facility",
    "hangar_outdoor": "industrial_facility",
    "dam": "industrial_facility",
    "excavation": "industrial_facility",
    "fire_escape": "industrial_facility",
    "landfill": "industrial_facility",
    "lock_chamber": "industrial_facility",
    "trench": "industrial_facility",
    "shed": "industrial_facility",

    # --- Urban outdoor ---
    "alley": "urban_outdoor",
    "building_facade": "urban_outdoor",
    "courtyard": "urban_outdoor",
    "crosswalk": "urban_outdoor",
    "downtown": "urban_outdoor",
    "plaza": "urban_outdoor",
    "street": "urban_outdoor",
    "shopfront": "urban_outdoor",
    "parking_lot": "urban_outdoor",
    "residential_neighborhood": "urban_outdoor",
    "skyscraper": "urban_outdoor",
    "amusement_park": "urban_outdoor",
    "arcade": "urban_outdoor",
    "balcony_exterior": "urban_outdoor",
    "bazaar_outdoor": "urban_outdoor",
    "boardwalk": "urban_outdoor",
    "driveway": "urban_outdoor",
    "doorway_outdoor": "urban_outdoor",
    "gazebo_exterior": "urban_outdoor",
    "market_outdoor": "urban_outdoor",
    "patio": "urban_outdoor",
    "plaza": "urban_outdoor",
    "promenade": "urban_outdoor",
    "rope_bridge": "urban_outdoor",
    "street": "urban_outdoor",
    "tower": "urban_outdoor",
    "slum": "urban_outdoor",
    "sky": "urban_outdoor",
    "roof_garden": "urban_outdoor",
    "phone_booth": "urban_outdoor",

    # --- Natural landscapes ---
    "badlands": "natural_landscape",
    "canyon": "natural_landscape",
    "cliff": "natural_landscape",
    "desert_sand": "natural_landscape",
    "desert_vegetation": "natural_landscape",
    "forest_broadleaf": "natural_landscape",
    "forest_path": "natural_landscape",
    "mountain": "natural_landscape",
    "mountain_path": "natural_landscape",
    "mountain_snowy": "natural_landscape",
    "valley": "natural_landscape",
    "volcano": "natural_landscape",
    "rock_arch": "natural_landscape",
    "rainforest": "natural_landscape",
    "bamboo_forest": "natural_landscape",
    "butte": "natural_landscape",
    "canal_natural": "natural_landscape",
    "cliff": "natural_landscape",
    "coast": "natural_landscape",
    "creek": "natural_landscape",
    "crevasse": "natural_landscape",
    "desert_road": "natural_landscape",
    "field_road": "natural_landscape",
    "forest_road": "natural_landscape",
    "grotto": "natural_landscape",
    "islet": "natural_landscape",
    "lawn": "natural_landscape",
    "marsh": "natural_landscape",
    "swamp": "natural_landscape",
    "vegetable_garden": "natural_landscape",
    "wheat_field": "natural_landscape",
    "wind_farm": "natural_landscape",
    "windmill": "natural_landscape",
    "zen_garden": "natural_landscape",
    "canal_urban": "natural_landscape",
    "fountain": "natural_landscape",

    # --- Cold environments ---
    "ice_floe": "cold_environment",
    "ice_shelf": "cold_environment",
    "iceberg": "cold_environment",
    "igloo": "cold_environment",
    "ski_resort": "cold_environment",
    "ski_slope": "cold_environment",
    "snowfield": "cold_environment",
    "tundra": "cold_environment",
    "glacier": "cold_environment",

    # --- Aquatic environments ---
    "aquarium": "aquatic_environment",
    "beach": "aquatic_environment",
    "boat_deck": "aquatic_environment",
    "boathouse": "aquatic_environment",
    "harbor": "aquatic_environment",
    "lake_natural": "aquatic_environment",
    "lagoon": "aquatic_environment",
    "ocean": "aquatic_environment",
    "pond": "aquatic_environment",
    "river": "aquatic_environment",
    "waterfall": "aquatic_environment",
    "swimming_pool_indoor": "aquatic_environment",
    "swimming_pool_outdoor": "aquatic_environment",
    "swimming_hole": "aquatic_environment",
    "hot_spring": "aquatic_environment",
    "beach_house": "aquatic_environment",
    "fishpond": "aquatic_environment",
    "moat_water": "aquatic_environment",
    "pier": "aquatic_environment",
    "raft": "aquatic_environment",
    "watering_hole": "aquatic_environment",
    "wave": "aquatic_environment",
    "water_tower": "aquatic_environment",
    "underwater_ocean_deep": "aquatic_environment",
    "lighthouse": "aquatic_environment",


    # --- Religious or historical sites ---
    "church_outdoor": "religious_or_historical_site",
    "mosque_outdoor": "religious_or_historical_site",
    "synagogue_outdoor": "religious_or_historical_site",
    "temple_asia": "religious_or_historical_site",
    "catacomb": "religious_or_historical_site",
    "mausoleum": "religious_or_historical_site",
    "palace": "religious_or_historical_site",
    "castle": "religious_or_historical_site",
    "ruin": "religious_or_historical_site",
    "arch": "religious_or_historical_site",
    "amphitheater": "religious_or_historical_site",
    "archaelogical_excavation": "religious_or_historical_site",
    "burial_chamber": "religious_or_historical_site",
    "cemetery": "religious_or_historical_site",
    "chalet": "religious_or_historical_site",
    "kasbah": "religious_or_historical_site",
    "medina": "religious_or_historical_site",
    "pagoda": "religious_or_historical_site",
    "throne_room": "religious_or_historical_site",

    # --- Sports and entertainment ---
    "arena_hockey": "sports_and_entertainment",
    "arena_performance": "sports_and_entertainment",
    "arena_rodeo": "sports_and_entertainment",
    "baseball_field": "sports_and_entertainment",
    "basketball_court_indoor": "sports_and_entertainment",
    "boxing_ring": "sports_and_entertainment",
    "football_field": "sports_and_entertainment",
    "gymnasium_indoor": "sports_and_entertainment",
    "martial_arts_gym": "sports_and_entertainment",
    "racecourse": "sports_and_entertainment",
    "stadium_football": "sports_and_entertainment",
    "stadium_soccer": "sports_and_entertainment",
    "stage_indoor": "sports_and_entertainment",
    "stage_outdoor": "sports_and_entertainment",
    "bowling_alley": "sports_and_entertainment",
    "movie_theater_indoor": "sports_and_entertainment",
    "amphitheater": "sports_and_entertainment",
    "athletic_field_outdoor": "sports_and_entertainment",
    "ball_pit": "sports_and_entertainment",
    "baseball_field": "sports_and_entertainment",
    "basketball_court_indoor": "sports_and_entertainment",
    "bow_window_indoor": "sports_and_entertainment",
    "boxing_ring": "sports_and_entertainment",
    "bullring": "sports_and_entertainment",
    "football_field": "sports_and_entertainment",
    "raceway": "sports_and_entertainment",
    "soccer_field": "sports_and_entertainment",
    "stadium_baseball": "sports_and_entertainment",
    "staircase": "sports_and_entertainment",
    "topiary_garden": "sports_and_entertainment",
    "water_park": "sports_and_entertainment",
    "carrousel": "sports_and_entertainment",
    "dining_hall": "sports_and_entertainment",
    "diner_outdoor": "sports_and_entertainment",
    "ice_skating_rink_indoor": "sports_and_entertainment",
    "ice_skating_rink_outdoor": "sports_and_entertainment",


    # --- Cultural spaces ---
    "art_studio": "cultural_space",
    "art_school": "cultural_space",
    "music_studio": "cultural_space",
    "television_studio": "cultural_space",
    "museum_outdoor": "cultural_space",
    "artists_loft": "cultural_space",
    "archive": "cultural_space",
    "galley": "cultural_space",
    "japanese_garden": "cultural_space",
    "natural_history_museum": "cultural_space",
    "music_studio": "cultural_space",
    "orchestra_pit": "cultural_space",

    # --- Rural or recreational area ---
    "aqueduct": "rural_or_recreational_area",
    "barn": "rural_or_recreational_area",
    "barndoor": "rural_or_recreational_area",
    "beer_garden": "rural_or_recreational_area",
    "beer_hall": "rural_or_recreational_area",
    "botanical_garden": "rural_or_recreational_area",
    "cabin_outdoor": "rural_or_recreational_area",
    "campsite": "rural_or_recreational_area",
    "campus": "rural_or_recreational_area",
    "farm": "rural_or_recreational_area",
    "field_cultivated": "rural_or_recreational_area",
    "field_wild": "rural_or_recreational_area",
    "golf_course": "rural_or_recreational_area",
    "greenhouse_outdoor": "rural_or_recreational_area",
    "hayfield": "rural_or_recreational_area",
    "orchard": "rural_or_recreational_area",
    "park": "rural_or_recreational_area",
    "pasture": "rural_or_recreational_area",
    "picnic_area": "rural_or_recreational_area",
    "playground": "rural_or_recreational_area",
    "rice_paddy": "rural_or_recreational_area",
    "sandbox": "rural_or_recreational_area",
    "stable": "rural_or_recreational_area",
    "tree_farm": "rural_or_recreational_area",
    "tree_house": "rural_or_recreational_area",
    "village": "rural_or_recreational_area",
    "vineyard": "rural_or_recreational_area",
    "volleyball_court_outdoor": "rural_or_recreational_area",
    "yard": "rural_or_recreational_area",
    "corn_field": "rural_or_recreational_area",
    "corral": "rural_or_recreational_area",
    "cottage": "rural_or_recreational_area",
    "formal_garden": "rural_or_recreational_area",
    "greenhouse_indoor": "rural_or_recreational_area",
    "hunting_lodge_outdoor": "rural_or_recreational_area",
    "inn_outdoor": "rural_or_recreational_area",
    "kennel_outdoor": "rural_or_recreational_area",
    "pavilion": "rural_or_recreational_area",
    "oast_house": "rural_or_recreational_area",
}


In [ ]:
set(places365_to_13categories.values())

{'aquatic_environment',
 'cold_environment',
 'cultural_space',
 'indoor_commercial',
 'indoor_institutional',
 'indoor_residential',
 'industrial_facility',
 'natural_landscape',
 'religious_or_historical_site',
 'rural_or_recreational_area',
 'sports_and_entertainment',
 'transport_infrastructure',
 'urban_outdoor'}

In [ ]:
# Now create train/test splits with clean data
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_test = df_train.reset_index(drop=True), df_test.reset_index(drop=True)
print("Training set shape: ",df_train.shape)
print("Test set shape: ",df_test.shape)

Training set shape:  (13135, 10)
Test set shape:  (3284, 10)


In [ ]:
face_processor = AutoImageProcessor.from_pretrained("trpakov/vit-face-expression", use_fast=True)
face_model = AutoModelForImageClassification.from_pretrained("trpakov/vit-face-expression")

face_detector = MTCNN()

scene_processor = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load pretrained Places365 ResNet18
scene_model = models.resnet18(num_classes=365)
weights_url = "http://places2.csail.mit.edu/models_places365/resnet18_places365.pth.tar"
checkpoint = torch.hub.load_state_dict_from_url(weights_url, map_location="cpu")
state_dict = {k.replace("module.", ""): v for k, v in checkpoint["state_dict"].items()}
scene_model.load_state_dict(state_dict)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# scene_model.to(device)

# face_model.to(device)

print("Loaded pretrained models")

Fetching 1 files: 100%|██████████| 1/1 [00:00<?, ?it/s]



Loaded pretrained models


In [ ]:
class TruncatedFaceModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        logits_list = []
        for face_tensor in x:
            face_output = self.model(face_tensor)
            face_logits = face_output.logits[:, :6]
            logits_list.append(face_logits)

        stacked = torch.cat(logits_list, dim=0)  # Stack along batch dimension
        return stacked.flatten()
    
face_model_truncated = TruncatedFaceModel(face_model)
face_network = Network(face_model_truncated, "face_model", batching = True)
face_network.optimizer = torch.optim.Adam(face_network.parameters(), lr=lr_faces)

In [ ]:
class SceneReductionModel(nn.Module):
    def __init__(self, input_dim=365, hidden_dim=128, output_dim=13):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)

scene_model_reduction = SceneReductionModel()
scene_reduction_network = Network(scene_model_reduction, "scene_reduction", batching = True)
scene_reduction_network.optimizer= torch.optim.Adam(scene_reduction_network.parameters(), lr=lr_scene_reduction)

In [ ]:
class TruncatedSceneModel(nn.Module):
    def __init__(self, model, scene_reduction_model):
        super().__init__()
        self.model = model
        self.scene_reduction_model = scene_reduction_model

    def forward(self, x):
        device = next(self.model.parameters()).device
        x = x.to(device)
        # x will be a tensor of the whole image (with the right preprocessing)
        scene_output = self.model(x)
        # Scene output has 365 values, we want it to be mapped to the 13 more common places
        # That mapping happens through a neural network that will be trained during DPL training
        scene_logits = self.scene_reduction_model(scene_output).squeeze(0)
        return scene_logits
    
scene_model_truncated = TruncatedSceneModel(scene_model, scene_model_reduction)
scene_network = Network(scene_model_truncated, "scene_model", batching = True)
scene_network.optimizer = torch.optim.Adam(scene_network.parameters(), lr=lr_scenes)

In [ ]:
class FindingEmoFaces(object):
    def __init__(self, df, subset, face_detector, max_size=2048):
        self.df = df
        self.face_detector = face_detector
        self.max_size = max_size
        self.subset = subset
        self.save_path = os.path.join(findingemo_dir, "precomputed", "detected_faces", subset)

    def __getitem__(self, item):
        index = int(item[0]) if isinstance(item, (tuple, list)) else int(item)
        # Load the tensor
        tensor = torch.load(os.path.join(self.save_path, f"{index}.pt"))
        return tensor

facetensors_train = FindingEmoFaces(df_train, "train", face_detector)
facetensors_test = FindingEmoFaces(df_test, "test", face_detector)

In [ ]:
class FindingEmoScene(object):
    def __init__(self, df):
        self.df = df

    def __getitem__(self, item):
        index = int(item[0]) if isinstance(item, (tuple, list)) else int(item)
        img_path = os.path.join(findingemo_dir, self.df.iloc[index]["image_path"])
        img = Image.open(img_path).convert("RGB")

        x = scene_processor(img).unsqueeze(0)
 
        return x
    
scenetensors_train = FindingEmoScene(df_train)
scenetensors_test = FindingEmoScene(df_test)

In [ ]:
x = scenetensors_train[0]
print(x.shape)
y = scene_model_truncated(x)
print(y)
print(y.shape)

torch.Size([1, 3, 224, 224])
tensor([-0.3154,  0.0289, -0.2803,  0.2321, -0.1322,  0.1622,  0.2603, -0.2541,
        -0.0596,  0.1122, -0.2275, -0.2385, -0.1646], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
torch.Size([13])


In [ ]:
x = facetensors_train[0]
print(x.shape)
y = face_model_truncated(x)
print(y)
print(y.shape)

torch.Size([3, 3, 224, 224])
DEBUG: face_batch shape before model: torch.Size([1, 3, 3, 224, 224])

DEBUG: face_batch shape before model: torch.Size([1, 3, 3, 224, 224])


ValueError: too many values to unpack (expected 4)

In [ ]:
model = Model(
    os.path.join(prolog_dir, "model3.pl"),
    [face_network, scene_reduction_network, scene_network]
)
engine = ExactEngine(model)
model.set_engine(engine, cache=False)
model.optimizer = SGD(model, param_lr=lr_model)

model.add_tensor_source("train_face", facetensors_train)
model.add_tensor_source("train_scene", scenetensors_train)
model.add_tensor_source("test_face", facetensors_test)
model.add_tensor_source("test_scene", scenetensors_test)

In [ ]:
class FindingEmo(Dataset, TorchDataset):
    def __init__(self, indices, df, subset_name):
        super(FindingEmo, self).__init__()
        self.df = df
        self.subset_name = subset_name
        self.data = indices

    def __len__(self):
        return len(self.data)
    
    def to_query(self, i):
        sample_idx = self.data[i]
        emotion_idx = self.df.iloc[sample_idx]['emotion_8_idx']

        E = Var("E")

        goal = Term("final_emo",
                    Term("tensor", Term(f"{self.subset_name}_face", Constant(sample_idx))),
                    Term("tensor", Term(f"{self.subset_name}_scene", Constant(sample_idx))),
                    E)
        
        return Query(goal, substitution={E: Constant(emotion_idx)})
        

In [ ]:
training_size = int(len(df_train) * training_size_perc)

train_indices = list(range(0, training_size))
test_indices = list(range(0,len(df_test)))

train_set = FindingEmo(train_indices, df_train, "train")
test_set = FindingEmo(test_indices, df_test, "test")

In [ ]:
# loader= DataLoader(train_set,batch_size=batch_size, shuffle=True)

In [ ]:
from problog.logic import Term, Constant, list2term
import torch.nn.functional as F

def softmax_to_list(tensor_term):
    """
    Softmax function that returns a Prolog list of probabilities
    Handles int (-1) error, device consistency, and tensor shape.
    """
    # If tensor_term is a Term, use engine.get_tensor
    if hasattr(tensor_term, "functor") and tensor_term.functor == "tensor":
        tensor = engine.get_tensor(tensor_term)
    # If tensor_term is an int, get directly from tensor_store
    elif isinstance(tensor_term, int):
        if tensor_term == -1:
            print("ERROR: softmax_to_list received -1 (tensor not found)")
            return list2term([])
        tensor = engine.tensor_store.get(tensor_term, None)
        if tensor is None:
            print(f"ERROR: tensor_store has no entry for {tensor_term}")
            return list2term([])
    else:
        tensor = tensor_term  # fallback, may be a tensor already

    if not isinstance(tensor, torch.Tensor):
        print(f"ERROR: softmax_to_list received non-tensor: {type(tensor)}")
        return list2term([])

    # Move tensor to CPU for softmax if needed
    if tensor.device.type != "cpu":
        tensor = tensor.cpu()

    # Flatten if needed
    if tensor.ndim == 0:
        tensor = tensor.unsqueeze(0)
    elif tensor.ndim > 1:
        tensor = tensor.view(-1)

    # Apply softmax
    probs = F.softmax(tensor, dim=-1)

    # Convert to Python list of floats
    prob_list = probs.tolist()

    # Convert to Prolog list format
    prolog_constants = [Constant(float(p)) for p in prob_list]
    prolog_list = list2term(prolog_constants)

    return prolog_list

# Register the function with your engine
engine.register_foreign(
    func=softmax_to_list,
    function_name="softmax",
    arity_in=1,   # Takes tensor input
    arity_out=1   # Returns Prolog list
)


In [ ]:
def debug_print(x):
    print("DEBUG Facelogits value:", x)
    print("Type:", type(x))
    if hasattr(x, "shape"):
        print("Shape:", x.shape)
    if isinstance(x, torch.Tensor):
        print("Tensor device:", x.device)
        print("Tensor dtype:", x.dtype)
        print("Tensor min:", x.min().item(), "max:", x.max().item())
        # Check device consistency with model
        try:
            import gc
            for obj in gc.get_objects():
                if hasattr(obj, 'parameters'):
                    try:
                        model_device = next(obj.parameters()).device
                        if x.device != model_device:
                            print(f"WARNING: Tensor device {x.device} does not match model device {model_device}")
                    except Exception:
                        pass
        except Exception as e:
            print("Device consistency check failed:", e)
    elif isinstance(x, int):
        print("WARNING: Value is int, likely an error (-1 means tensor not found)")
    else:
        print("WARNING: Unexpected type for debug_print")
    return ()

engine.register_foreign(
    func=debug_print,
    function_name="debug_print",
    arity_in=1,
    arity_out=0
)

In [ ]:
print(facetensors_train[0].shape)

torch.Size([3, 3, 224, 224])


In [ ]:
x = facetensors_train[0]
print(x.shape)
y = face_model_truncated(x)
print(y)
print(y.shape)

torch.Size([3, 3, 224, 224])
DEBUG: face_batch shape before model: torch.Size([3, 3, 224, 224])
tensor([-0.2959, -2.5035,  0.5354, -0.8110,  4.0392, -1.5357, -1.0524, -2.2824,
        -0.5259, -2.4327,  2.8070,  3.7185, -0.6956, -1.1049,  1.4585, -2.2307,
        -0.8984,  4.0905], device='cuda:0')
torch.Size([18])


In [ ]:
x = scenetensors_train[0]
print(x.shape)

torch.Size([1, 3, 224, 224])


In [ ]:
print("Available train_face keys:", list(model.tensor_sources["train_face"].__dict__.keys()))

Available train_face keys: ['df', 'face_detector', 'max_size', 'subset', 'save_path']


In [ ]:
x = facetensors_train[0]
print("Manual tensor fetch shape:", x.shape)

Manual tensor fetch shape: torch.Size([3, 3, 224, 224])


In [ ]:
# Python
from problog.logic import Term, Constant, Var
from deepproblog.query import Query

goal = Term("face_emotion_prob", Term("tensor", Term("train_face", Constant(0))), Var("E"))
query = Query(goal, substitution={})
results = model.solve([query])
print("DPL result:", results[0])

DEBUG Facelogits value: anger
Type: <class 'problog.logic.Term'>
DEBUG Facelogits value: disgust
Type: <class 'problog.logic.Term'>
DEBUG Facelogits value: fear
Type: <class 'problog.logic.Term'>
DEBUG Facelogits value: joy
Type: <class 'problog.logic.Term'>
DEBUG Facelogits value: sadness
Type: <class 'problog.logic.Term'>
DEBUG Facelogits value: surprise
Type: <class 'problog.logic.Term'>
DEBUG: face_batch shape before model: torch.Size([3, 3, 224, 224])


IndexError: invalid index of a 0-dim tensor. Use `tensor.item()` in Python or `tensor.item<T>()` in C++ to convert a 0-dim tensor to a number